# **grafo.py**

In [12]:
from collections import defaultdict, deque
import math
import pandas as pd
 #------------Definição do grafo-------------
class Grafo:
    def __init__(self):
        self.veiculos = 0;
        self.capacidade_veiculos = 0;
        self.no_deposito = None;
        self.vertices = set()
        self.nos_requeridos = {}

        self.arestas_requeridas = []
        self.arcos_requeridos = []

        self.arestas_nao_requeridas = []
        self.arcos_nao_requeridos = []

    def adicionar_no_requerido(self, id_no, demanda, custo_servico, id_servico):
        self.nos_requeridos[id_no] = {
            "demanda": demanda,
            "custo_servico": custo_servico,
            "id_servico": id_servico
        }
        self.vertices.add(id_no)

    def adicionar_aresta_requerida(self, de, para, custo_transporte, demanda, custo_servico, id_servico):
        self.arestas_requeridas.append({
            "de": de,
            "para": para,
            "custo_transporte": custo_transporte,
            "demanda": demanda,
            "custo_servico": custo_servico,
            "id_servico": id_servico
        })
        self.vertices.update([de, para])

    def adicionar_arco_requerido(self, de, para, custo_transporte, demanda, custo_servico, id_servico):
        self.arcos_requeridos.append({
            "de": de,
            "para": para,
            "custo_transporte": custo_transporte,
            "demanda": demanda,
            "custo_servico": custo_servico,
            "id_servico": id_servico
        })
        self.vertices.update([de, para])

    def adicionar_aresta_nao_requerida(self, de, para, custo_transporte):
        self.arestas_nao_requeridas.append({
            "de": de,
            "para": para,
            "custo_transporte": custo_transporte
        })
        self.vertices.update([de, para])

    def adicionar_arco_nao_requerido(self, de, para, custo_transporte):
        self.arcos_nao_requeridos.append({
            "de": de,
            "para": para,
            "custo_transporte": custo_transporte
        })
        self.vertices.update([de, para])

#------------Definição do grafo---------------------



#-------------Calculos Estatísticos------------------
    def total_vertices(self):
        return len(self.vertices)

    def total_arestas(self):
        return len(self.arestas_requeridas) + len(self.arestas_nao_requeridas)

    def total_arcos(self):
        return len(self.arcos_requeridos) + len(self.arcos_nao_requeridos)

    def total_vertices_requeridos(self):
        return len(self.nos_requeridos)

    def total_arestas_requeridas(self):
        return len(self.arestas_requeridas)

    def total_arcos_requeridos(self):
        return len(self.arcos_requeridos)

    def densidade(self):
        n = self.total_vertices()
        e = self.total_arestas() + self.total_arcos()
        if n <= 1:
            return 0
        return e / (n * (n - 1))

    def construir_grafo_nao_direcionado(self):
        grafo = defaultdict(list)
        for a in self.arestas_requeridas + self.arestas_nao_requeridas:
            grafo[a["de"]].append(a["para"])
            grafo[a["para"]].append(a["de"])
        for a in self.arcos_requeridos + self.arcos_nao_requeridos:
            grafo[a["de"]].append(a["para"])
        return grafo

    def componentes_conectados(self):
        grafo = self.construir_grafo_nao_direcionado()
        visitados = set()
        componentes = 0

        for v in self.vertices:
            if v not in visitados:
                componentes += 1
                fila = deque([v])
                while fila:
                    atual = fila.popleft()
                    if atual in visitados:
                        continue
                    visitados.add(atual)
                    fila.extend(grafo[atual])
        return componentes

    def grau_dos_vertices(self):
        graus = defaultdict(int)
        for a in self.arestas_requeridas + self.arestas_nao_requeridas:
            graus[a["de"]] += 1
            graus[a["para"]] += 1
        for a in self.arcos_requeridos + self.arcos_nao_requeridos:
            graus[a["de"]] += 1
        return graus

    def grau_minimo(self):
        graus = self.grau_dos_vertices()
        return min(graus.values()) if graus else 0

    def grau_maximo(self):
        graus = self.grau_dos_vertices()
        return max(graus.values()) if graus else 0

    def matriz_adjacencia_pesos(self):
        n = max(self.vertices) + 1
        dist = [[math.inf] * n for _ in range(n)]
        for i in self.vertices:
            dist[i][i] = 0
        for a in self.arestas_requeridas + self.arestas_nao_requeridas:
            dist[a["de"]][a["para"]] = a["custo_transporte"]
            dist[a["para"]][a["de"]] = a["custo_transporte"]
        for a in self.arcos_requeridos + self.arcos_nao_requeridos:
            dist[a["de"]][a["para"]] = a["custo_transporte"]
        return dist

    def floyd_warshall(self):
        dist = self.matriz_adjacencia_pesos()
        n = len(dist)
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if dist[i][k] + dist[k][j] < dist[i][j]:
                        dist[i][j] = dist[i][k] + dist[k][j]
        return dist

    def caminho_medio(self):
        dist = self.floyd_warshall()
        total = 0
        cont = 0
        for i in self.vertices:
            for j in self.vertices:
                if i != j and dist[i][j] < math.inf:
                    total += dist[i][j]
                    cont += 1
        return total / cont if cont > 0 else 0

    def diametro(self):
        dist = self.floyd_warshall()
        maior = 0
        for i in self.vertices:
            for j in self.vertices:
                if i != j and dist[i][j] < math.inf:
                    maior = max(maior, dist[i][j])
        return maior

    def intermediacao(self):
        dist = self.floyd_warshall()
        intermed = defaultdict(float)
        for s in self.vertices:
            for t in self.vertices:
                if s == t:
                    continue
                caminho_min = dist[s][t]
                for v in self.vertices:
                    if v != s and v != t:
                        if dist[s][v] + dist[v][t] == caminho_min:
                            intermed[v] += 1
        return intermed
#-------------Calculos Estatísticos------------------


# **manipularArquivo.py**

In [13]:
#from grafo import Grafo

#-------------Leitura do Arquivo------------------
def ler_grafo_de_arquivo(caminho_arquivo):
    grafo = Grafo()
    lendo_nos = lendo_arestas_req = lendo_arcos_req = lendo_arestas_nreq = lendo_arcos_nreq = False
    cabecalhos = ["FROM", "TO", "T.", "DEMAND", "S.", "N."]

    id_servico =1 #zero é reservado para depósito

    with open(caminho_arquivo, 'r') as arquivo:
        for linha in arquivo:
            linha = linha.strip()
            if linha == "":
                continue
            if linha.startswith("#Vehicles:"):
                grafo.veiculos = int(linha.split(":")[1].strip())
                continue
            elif linha.startswith("Depot Node"):
                grafo.no_deposito = int(linha.split(":")[1].strip())
                continue
            elif linha.startswith("Capacity"):
                grafo.capacidade_veiculos = int(linha.split(":")[1].strip())
                continue
            elif linha.startswith("ReN."):
                lendo_nos = True
                lendo_arestas_req = lendo_arcos_req = lendo_arestas_nreq = lendo_arcos_nreq = False
                continue
            elif linha.startswith("ReE."):
                lendo_arestas_req = True
                lendo_nos = lendo_arcos_req = lendo_arestas_nreq = lendo_arcos_nreq = False
                continue
            elif linha.startswith("ReA."):
                lendo_arcos_req = True
                lendo_nos = lendo_arestas_req = lendo_arestas_nreq = lendo_arcos_nreq = False
                continue
            elif linha.startswith("EDGE"):
                lendo_arestas_nreq = True
                lendo_nos = lendo_arestas_req = lendo_arcos_req = lendo_arcos_nreq = False
                continue
            elif linha.startswith("ARC"):
                lendo_arcos_nreq = True
                lendo_nos = lendo_arestas_req = lendo_arcos_req = lendo_arestas_nreq = False
                continue

            if any(p in linha.upper() for p in cabecalhos):
                continue

            partes = linha.split()
            if lendo_nos and linha.startswith("N"):
                grafo.adicionar_no_requerido(int(partes[0][1:]), int(partes[1]), int(partes[2]), id_servico)
                id_servico += 1
            elif lendo_arestas_req and linha.startswith("E"):
                grafo.adicionar_aresta_requerida(int(partes[1]), int(partes[2]), int(partes[3]), int(partes[4]), int(partes[5]), id_servico)
                id_servico += 1
            elif lendo_arcos_req and linha.startswith("A"):
                grafo.adicionar_arco_requerido(int(partes[1]), int(partes[2]), int(partes[3]), int(partes[4]), int(partes[5]), id_servico)
                id_servico += 1
            elif lendo_arestas_nreq and linha.startswith("NrE"):
                grafo.adicionar_aresta_nao_requerida(int(partes[1]), int(partes[2]), int(partes[3]))
            elif lendo_arcos_nreq and linha.startswith("NrA"):
                grafo.adicionar_arco_nao_requerido(int(partes[1]), int(partes[2]), int(partes[3]))
    return grafo
#-------------Leitura do Arquivo------------------------


#-------------Gerar csv---------------------------------
def exportar_estatisticas_para_csv(grafo, nome_arquivo):
    dados = {
        "Numero de veículo": grafo.veiculos,
        "Capacidade de veículo": grafo.capacidade_veiculos,
        "No deposito": grafo.no_deposito,
        "Total de vértices": grafo.total_vertices(),
        "Total de arestas": grafo.total_arestas(),
        "Total de arcos": grafo.total_arcos(),
        "Total de vértices requeridos": grafo.total_vertices_requeridos(),
        "Total de arestas requeridas": grafo.total_arestas_requeridas(),
        "Total de arcos requeridos": grafo.total_arcos_requeridos(),
        "Densidade": grafo.densidade(),
        "Componentes conectados": grafo.componentes_conectados(),
        "Grau mínimo": grafo.grau_minimo(),
        "Grau máximo": grafo.grau_maximo(),
        "Caminho médio": grafo.caminho_medio(),
        "Diâmetro": grafo.diametro()
    }

    df = pd.DataFrame(dados.items(), columns=["Estatística", "Valor"])
    df.to_csv(nome_arquivo, index=False)
    print(f"\nEstatísticas exportadas para '{nome_arquivo}' com sucesso.")


#-------------Gerar csv---------------------------------

# **Rotas**

In [14]:
import math

def construir_rotas(grafo):
    distancias = grafo.floyd_warshall()
    servicos_pendentes = []

    # Generaliza todos os serviços requeridos (nós, arestas, arcos)
    for no, dados in grafo.nos_requeridos.items():
        servicos_pendentes.append({
            "tipo": "nó",
            "de": no,
            "para": no,
            "demanda": dados["demanda"],
            "custo_servico": dados["custo_servico"],
            "id_servico": dados["id_servico"]
        })

    for a in grafo.arestas_requeridas:
        servicos_pendentes.append({
            "tipo": "aresta",
            "de": a["de"],
            "para": a["para"],
            "demanda": a["demanda"],
            "custo_servico": a["custo_servico"],
            "id_servico": a["id_servico"]
        })

    for a in grafo.arcos_requeridos:
        servicos_pendentes.append({
            "tipo": "arco",
            "de": a["de"],
            "para": a["para"],
            "demanda": a["demanda"],
            "custo_servico": a["custo_servico"],
            "id_servico": a["id_servico"]
        })

    demanda_total_esperada = sum(s["demanda"] for s in servicos_pendentes)
    print(f"\nDemanda total esperada: {demanda_total_esperada}\n")

    rotas = []
    rota_numero = 1

    while servicos_pendentes:
        carga_atual = 0
        custo_total = 0
        rota = [grafo.no_deposito]
        posicao_atual = grafo.no_deposito
        servicos_da_rota = []
        servicos_restantes = []

        print(f"Demanda da rota {rota_numero}:")

        for s in servicos_pendentes:
            if s["tipo"] == "nó":
                destinos = [s["de"]]
            else:
                destinos = [s["de"], s["para"]]

            custo_transporte = 0
            pos = posicao_atual
            caminho_valido = True

            for d in destinos:
                if distancias[pos][d] == math.inf:
                    caminho_valido = False
                    break
                custo_transporte += distancias[pos][d]
                pos = d

            if not caminho_valido:
                servicos_restantes.append(s)
                continue

            if carga_atual + s["demanda"] > grafo.capacidade_veiculos:
                servicos_restantes.append(s)
                continue

            print(f"  Serviço {s['id_servico']} ({s['tipo']}) de {s['de']} para {s['para']} - Demanda: {s['demanda']}")

            carga_atual += s["demanda"]
            custo_total += custo_transporte + s["custo_servico"]

            for d in destinos:
                if rota[-1] != d:
                    rota.append(d)

            posicao_atual = destinos[-1]
            servicos_da_rota.append(s)

        servicos_pendentes = servicos_restantes

        if len(rota) > 1:
            custo_volta = distancias[posicao_atual][grafo.no_deposito]
            if custo_volta < math.inf:
                custo_total += custo_volta
                rota.append(grafo.no_deposito)
            else:
                print(f"  [!] Não foi possível retornar do nó {posicao_atual} para o depósito {grafo.no_deposito}")

            rotas.append({
                "rota": rota,
                "custo_total": custo_total,
                "demanda_total": carga_atual,
                "servicos_atendidos": servicos_da_rota
            })
            print(f"  Total da rota {rota_numero}: {carga_atual}\n")
        else:
            print(f"  [!] Rota {rota_numero} não teve serviços atendidos\n")

        rota_numero += 1

    demanda_total_calculada = sum(r["demanda_total"] for r in rotas)
    print(f"Demanda total nas rotas: {demanda_total_calculada}\n")

    return rotas


# **Busca Local**

In [15]:
import time
import copy

def calcular_custo_rota(grafo, servicos):
    dist = grafo.floyd_warshall()
    deposito = grafo.no_deposito
    rota = [deposito]
    posicao = deposito
    custo_total = 0
    demanda_total = 0

    for s in servicos:
        destino = s["de"]
        custo_total += dist[posicao][destino]
        custo_total += s["custo_servico"]
        demanda_total += s["demanda"]
        rota.append(destino)
        posicao = destino

    if posicao != deposito:
        custo_total += dist[posicao][deposito]
        rota.append(deposito)

    return {
        "rota": rota,
        "custo_total": custo_total,
        "demanda_total": demanda_total,
        "servicos_atendidos": servicos
    }


def busca_local(grafo, rotas, tempo_max_segundos=250, iteracoes_sem_melhora_limite=7):
    inicio = time.time()
    iteracoes_sem_melhora = 0
    dist = grafo.floyd_warshall()

    while iteracoes_sem_melhora < iteracoes_sem_melhora_limite and (time.time() - inicio) < tempo_max_segundos:
        melhorou = False

        for i in range(len(rotas)):
            for j in range(len(rotas)):
                if i == j:
                    continue

                rota_i = rotas[i]
                rota_j = rotas[j]

                for s in rota_i["servicos_atendidos"]:
                    if rota_j["demanda_total"] + s["demanda"] <= grafo.capacidade_veiculos:
                        nova_servicos_i = [ss for ss in rota_i["servicos_atendidos"] if ss != s]
                        nova_servicos_j = rota_j["servicos_atendidos"] + [s]

                        nova_rota_i = calcular_custo_rota(grafo, nova_servicos_i)
                        nova_rota_j = calcular_custo_rota(grafo, nova_servicos_j)

                        custo_antigo = rota_i["custo_total"] + rota_j["custo_total"]
                        custo_novo = nova_rota_i["custo_total"] + nova_rota_j["custo_total"]

                        if custo_novo < custo_antigo:
                            rotas[i] = nova_rota_i
                            rotas[j] = nova_rota_j
                            melhorou = True
                            break
                if melhorou:
                    break
            if melhorou:
                break

        if melhorou:
            iteracoes_sem_melhora = 0
        else:
            iteracoes_sem_melhora += 1

    return rotas


# **vizualizacao**

In [ ]:
# visualizacao_grafo.ipynb
#from main import Grafo, ler_grafo_de_arquivo, exportar_estatisticas_para_csv
import time
import math
import os

# Solicita o caminho completo do diretório de entrada
diretorio_entrada = input("Digite o caminho do diretório com os arquivos .dat: ").strip()

#Inicio da contagem de Clock (clock global)
inicio = time.perf_counter()

# Diretórios de saída
diretorio_saida_rotas = "drive/MyDrive/ProjetoGrafos/G16-Sol"
diretorio_saida_estatisticas = "drive/MyDrive/ProjetoGrafos/G16-estatisticas"

os.makedirs(diretorio_saida_rotas, exist_ok=True)
os.makedirs(diretorio_saida_estatisticas, exist_ok=True)

# Processa todos os arquivos .dat do diretório informado
for nome_arquivo in os.listdir(diretorio_entrada):
    if nome_arquivo.endswith(".dat"):
        caminho_entrada = os.path.join(diretorio_entrada, nome_arquivo)
        nome_sem_extensao = os.path.splitext(nome_arquivo)[0]
        caminho_saida_rotas = os.path.join(diretorio_saida_rotas, f"sol-{nome_sem_extensao}.dat")
        caminho_saida_estatisticas = os.path.join(diretorio_saida_estatisticas, f"{nome_sem_extensao}.csv")

        print(f"\n--- Processando: {nome_arquivo} ---")
        inicio = time.perf_counter()

        grafo = ler_grafo_de_arquivo(caminho_entrada)

        # Exportar estatísticas do grafo
        exportar_estatisticas_para_csv(grafo, caminho_saida_estatisticas)

        # Cálculo das rotas
        rotas = construir_rotas(grafo)
        #pontoDeRotas = time.perf_counter()

        # Busca Local
        rotas = busca_local(grafo, rotas)
        pontoDeRotas = time.perf_counter()

        # Cálculo dos clocks
        tempoMS = pontoDeRotas - inicio
        fimPrograma = time.perf_counter()
        tempoFP = fimPrograma - inicio

        # Gravação do arquivo de saída de rotas
        with open(caminho_saida_rotas, "w") as arquivo_saida:
            print(f"{sum(r['custo_total'] for r in rotas)}", file=arquivo_saida)
            print(f"{len(rotas)}", file=arquivo_saida)
            print(f"{int(tempoFP * 3.0 * 1e9)}", file=arquivo_saida)
            print(f"{int(tempoMS * 3.0 * 1e9)}", file=arquivo_saida)
            for i, r in enumerate(rotas):
                linha = f" 0 1 {i+1} {r['demanda_total']} {r['custo_total']}  {len(r['servicos_atendidos'])}"
                if r["rota"][0] == grafo.no_deposito:
                    linha += f" (D 0,{grafo.no_deposito},{grafo.no_deposito})"
                for s in r["servicos_atendidos"]:
                    linha += f" (S {s['id_servico']},{s['de']},{s['para']})"
                if r["rota"][-1] == grafo.no_deposito:
                    linha += f" (D 0,{grafo.no_deposito},{grafo.no_deposito})"
                print(linha, file=arquivo_saida)

        print(f"-> Arquivo de rotas salvo em: '{caminho_saida_rotas}'")
        print(f"-> Estatísticas exportadas para: '{caminho_saida_estatisticas}'")


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-17-2781722981.py", line 41, in <cell line: 0>
    rotas = busca_local(grafo, rotas)
            ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-15-3261157722.py", line 54, in busca_local
    nova_rota_j = calcular_custo_rota(grafo, nova_servicos_j)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-15-3261157722.py", line 5, in calcular_custo_rota
    dist = grafo.floyd_warshall()
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-12-1116884318.py", line None, in floyd_warshall
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb 